In [227]:
import get_data
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import pandas as pd
from random import random

# Single Stock Example

hyperparameters to tune: `rec_days`, `pool_num`
variablesto add: drop days??
for AAPL, 30 recovery days is best, while for XOM, 10 is best

We first get all the relevant data. We care about adjusted close and all technical indicators. The variables `drop_days`, `drop_percent`, and `rec_days` are used for the label and selecting rows for the dataset. BUT FIRST pick start and end dates.

In [234]:
company = "XOM"

all_data = get_data.get_data(company).sort_values('date').drop(["1. open", "2. high", "3. low", "4. close", "6. volume", "7. dividend amount", "8. split coefficient"], axis=1)
all_data['recovered'] = 0
all_data['drop'] = 0
all_data['in range'] = 0
all_data['test'] = [1 if random() < 0.25 else 0 for n in range(len(all_data))] # usually set to 0.25
price_data = all_data['5. adjusted close']

start_ind = all_data.index.get_loc("2009-12-31")
print("Training will start " + str(all_data.index[start_ind]))

Training will start 2009-12-31 00:00:00


In [235]:
final_set = []
rec_days = 10
end_ind = all_data.index.get_loc("2018-01-02") # train with data until the end of 2018. 2019-2020 will be used as simulation.
print("Training will end " + str(all_data.index[end_ind]))

drops = 0
recs = 0
for n in range(start_ind, end_ind):
    date = all_data.index[n]
    all_data['in range'][n] = 1
    
    if all_data["5. adjusted close"][n-1] > all_data["5. adjusted close"][n]:
        all_data['drop'][n] = 1
        drops += 1
        if all_data["5. adjusted close"][n + rec_days] > all_data["5. adjusted close"][n]:
            recs += 1
            all_data['recovered'][n] = 1


train_data = all_data[all_data['in range'] == 1]
train_data = all_data[all_data['drop'] == 1]
train_data = train_data.drop(columns=['drop', 'in range', '5. adjusted close'])
assert(len(train_data) == drops)

Training will end 2018-01-02 00:00:00


C:\Users\yugan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\yugan\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\yugan\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Training
We train the model. First, divide the training data into training and test data. Then preprocess.

In [236]:
test_data = train_data[train_data['test'] == 1].drop(columns=['test'])
train_data = train_data[train_data['test'] == 0].drop(columns=['test'])
print(len(test_data), len(train_data))

237 755


Train the model. And test it on the test_data created earlier.

In [237]:
features = train_data.columns[:-1] # everything but 'recovered' is a feature
y = train_data['recovered']

# rfc = RandomForestClassifier(n_jobs=-1, n_estimators = 1000)
rfc = GradientBoostingClassifier(n_estimators = 1000)
rfc.fit(train_data[features], y)
predictions = rfc.predict(test_data[features])
assert(len(predictions) == len(test_data))

eval = []
for n in range(len(predictions)):
    eval.append(int(predictions[n] == test_data['recovered'][n]))
print(str(sum(eval) / len(eval)) + " accuracy")

0.7679324894514767 accuracy


Now that we know it works, retrain with most of the data -- go back to 2nd code cell and change `all_data['test']` to  `[1 if random() < 0.01 else 0 for n in range(len(all_data))]`

## Simulating buying

In [238]:
pools = 15
principal = 2000

In [239]:
def sim(model, features, all_data, start_ind, end_ind, recovery, pool_num, principal, wallet_factor):
    '''
    Notes:
    start_ind should be the old end_ind, since the training data goes up to end_ind.
    end_ind should be current day
    model should be fit
    '''
    wallet = 0
    # [[money, in_use, days_from_buy]] :
    virtual_wallet = [[principal / pool_num, False, 0] for n in range(pool_num)]
    pool_counter = 0
    
    for n in range(start_ind, end_ind):
        date = all_data.index[n]
        if n % 20 == 0:
            print(date)
        # beginning of day actions -- sell a stock if it's reached its recovery time
        pools_taken = True
        for pool in virtual_wallet:
            pools_taken = pools_taken * pool[1]
            if pool[1]:
                pool[2] += 1
            if pool[2] == recovery:
                # SELL SELL SELL, reset the pool
                pool[1] = False
                pool[2] = 0
                buy_price = all_data['5. adjusted close'][n - recovery] # should be the "today_price"
                sell_price = all_data['5. adjusted close'][n]
                pool[0] = pool[0] + (pool[0] / buy_price) * (sell_price - buy_price)
                if pool[0] > (principal / pool_num): # if the pool is bigger than the original amount:
                    wallet += (pool[0] - (principal / pool_num)) * wallet_factor
                    pool[0] = pool[0] - ((pool[0] - (principal / pool_num)) * wallet_factor)
                if pool[0] < 1:
                    print("your pool died")
                    raise
        if pools_taken == 1: # no point doing anything if there are no pools available to invest from
            continue
            
        # check if a purchase should be made for the day
        today_data = all_data[features].iloc[[n]]
        if model.predict(today_data) == 1:
            # BUY BUY BUY
            while virtual_wallet[pool_counter][1]: # go to the next available pool
                pool_counter += 1
                pool_counter = pool_counter % pool_num

            virtual_wallet[pool_counter][1] = True # this pool is in_use
            pool_counter += 1
            pool_counter = pool_counter % pool_num
            
    return(wallet, virtual_wallet)

w, vw = sim(rfc, features, all_data, end_ind, all_data.index.get_loc("2020-01-31"), rec_days, pools, principal, 1)
total_money_made = 0
for pool in vw:
    print(pool[0])
    total_money_made += pool[0]
print(w, total_money_made)

2018-01-02 00:00:00
2018-01-31 00:00:00
2018-03-01 00:00:00
2018-03-29 00:00:00
2018-04-27 00:00:00
2018-05-25 00:00:00
2018-06-25 00:00:00
2018-07-24 00:00:00
2018-08-21 00:00:00
2018-09-19 00:00:00
2018-10-17 00:00:00
2018-11-14 00:00:00
2018-12-14 00:00:00
2019-01-15 00:00:00
2019-02-13 00:00:00
2019-03-14 00:00:00
2019-04-11 00:00:00
2019-05-10 00:00:00
2019-06-10 00:00:00
2019-07-09 00:00:00
2019-08-06 00:00:00
2019-09-04 00:00:00
2019-10-02 00:00:00
2019-10-30 00:00:00
2019-11-27 00:00:00
2019-12-27 00:00:00
2020-01-28 00:00:00
127.78969143564906
122.10256085413079
121.59350031103138
132.13372201392892
125.4999994266611
128.59550291299533
123.20432533686098
103.63697410912931
108.85045538056458
107.82354353898444
111.17790055000049
123.67534733855445
126.1514007585079
125.32307607452506
132.93344209560908
272.3408133198127 1820.491442137133
